In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def compute_energy(image, depth_map=None, saliency_map=None):
    """Compute the energy map of the image with optional depth/saliency maps."""
    gray = cv2.cvtColor((image*255).astype(np.uint8), cv2.COLOR_BGR2GRAY).astype(np.float32)
    
    # Gradient-based energy
    grad_x = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
    energy = np.abs(grad_x) + np.abs(grad_y)

    # Incorporate depth map
    if depth_map is not None:
        energy += depth_map * np.max(energy)

    # Incorporate saliency map
    if saliency_map is not None:
        energy += saliency_map * np.max(energy)

    return energy

def find_vertical_seam(energy):
    """Find the lowest-energy vertical seam using dynamic programming."""
    h, w = energy.shape
    M = energy.copy()
    backtrack = np.zeros_like(M, dtype=np.int32)

    for i in range(1, h):
        for j in range(w):
            if j == 0:
                idx = np.argmin(M[i-1, j:j+2])
                backtrack[i, j] = idx + j
                min_energy = M[i-1, idx + j]
            elif j == w-1:
                idx = np.argmin(M[i-1, j-1:j+1])
                backtrack[i, j] = idx + j - 1
                min_energy = M[i-1, idx + j - 1]
            else:
                idx = np.argmin(M[i-1, j-1:j+2])
                backtrack[i, j] = idx + j - 1
                min_energy = M[i-1, idx + j - 1]
            M[i, j] += min_energy

    # Reconstruct seam path
    seam = np.zeros(h, dtype=np.int32)
    seam[-1] = np.argmin(M[-1])
    for i in reversed(range(h-1)):
        seam[i] = backtrack[i+1, seam[i+1]]

    seam_energy = M[-1, seam[-1]]
    return seam, seam_energy

def remove_vertical_seam(image, seam):
    """Remove a vertical seam from the image."""
    h, w, c = image.shape
    output = np.zeros((h, w-1, c), dtype=image.dtype)
    for i in range(h):
        output[i] = np.delete(image[i], seam[i], axis=0)
    return output

def visualize_seam(image, seam):
    """Visualize the seam on the image."""
    img_seam = image.copy()
    for i in range(len(seam)):
        img_seam[i, seam[i]] = [1.0, 0, 0]  # Red color (float image)
    plt.imshow(img_seam)
    plt.axis('off')
    plt.show()

def seam_carving(image, seams_number, depth_map=None, saliency_map=None, visualize=False):
    """Perform Seam Carving to reduce image width."""
    resized_image = image.copy()
    seam_energies = []

    for i in range(seams_number):
        energy = compute_energy(resized_image, depth_map, saliency_map)
        seam, seam_energy = find_vertical_seam(energy)
        seam_energies.append(seam_energy)

        if visualize:
            visualize_seam(resized_image, seam)

        resized_image = remove_vertical_seam(resized_image, seam)

        # Update depth and saliency maps if provided
        if depth_map is not None:
            depth_map = remove_vertical_seam(depth_map[..., np.newaxis], seam)[..., 0]
        if saliency_map is not None:
            saliency_map = remove_vertical_seam(saliency_map[..., np.newaxis], seam)[..., 0]

    return resized_image, seam_energies


# -------------------------------
# Load images and maps
baby_path = 'Snowman.png'
baby_dmap_path = 'Snowman_DMap.png'
baby_smap_path = 'Snowman_SMap.png'

baby = cv2.imread(baby_path).astype(np.float32) / 255.0
baby_dmap = cv2.imread(baby_dmap_path, cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.0
baby_smap = cv2.imread(baby_smap_path, cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.0

# Parameters
seams_number = 200
visualize = False

# Process image
baby_resized, baby_energies = seam_carving(baby, seams_number, baby_dmap, baby_smap, visualize)

# Save seam energies
with open('seam_energy_log.txt', 'w') as f:
    f.write('Baby Seam Energies:\n' + '\n'.join(map(str, baby_energies)) + '\n')

# Visualize result
plt.imshow(cv2.cvtColor((baby_resized*255).astype(np.uint8), cv2.COLOR_BGR2RGB))
plt.title('Baby Resized')
plt.axis('off')
plt.show()

# Save resized image
cv2.imwrite('Baby_resized.png', (baby_resized*255).astype(np.uint8))


AttributeError: 'NoneType' object has no attribute 'astype'